In [ ]:
%matplotlib inline
import os
import sys
import yaml
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO
import cv2


#get custom functions to use
sys.path.append('../')
from myutils import get_video_info, capture_frames, train_valid_test_split
from myutils import track_droplet
from plot_utils import add_speed, plot_speed

## Create Particle Detection Dataset

- Some global variables we will keep using in this tutorial are below. We will demonstrate everything on the walking droplet experiment with 3 walkers. You should be able to train your own model by following the same steps for your own experiment


- **To create your own model using this tutorial, make sure to create your own folder instead of working on "tutorial_data" folder. Otherwise, your results will overwrite on the actual repository data**.

In [ ]:
#three-droplet experiment video
video_path = "../dataset/videos/three_droplet.mp4"

Assuming you have the experiment video, first step is to save some sample frames from the video source. It is always a good idea to capture only the relevant components of the experiment such as experiment corral and walkers etc. Thus you may want to crop your video before creating training data. You can use [Avidemux](https://avidemux.sourceforge.net/download.html0) which is an amazing online tool. First call **"get_video_info"** helper function to get an idea about the video source.

In [ ]:
get_video_info(video_path)

Based on the info above, decide how many frames to be captured. For example, if we aim for 180 frames in total, we need to capture one frame in every (total_frames/180)/frame_rate = 1.7 second. **"capture_frames"** function does this job and captures one frame in every *save_interval* second and save to *image_dir*. Make sure you are happy with the captures images in *image_dir*.

In [ ]:
#save dir for captures frames
image_dir = "raw_images/"
capture_frames(video_path=video_path,image_dir = image_dir,save_interval=1.7)

We will now create a traning/validation/testing dataset from these frames. The most common ratio is 70/20/10. **train_valid_test_split** helper function does this job. This function creates a folder structure for train/valid/test data as 

                    dest_dir/datasets/train/images and dest_dir/datasets/train/labels
                    dest_dir/datasets/valid/images and dest_dir/datasets/valid/labels 
                    dest_dir/datasets/test/images and dest_dir/datasets/test/labels 

At this point, *labels* folders are empty. The same function also creates  *sample.yaml and dummy_test.yaml* files inside * dest_dir/datasets*
that are necessary for traninng and testing our YOLO models. Make sure these files are all in place. "datasets" folder is there due to an ongoing bug in YOLOv8, not a big deal. 

In [ ]:
train_valid_test_split(image_dir=image_dir,dest_dir='./',
                       train_ratio=0.7, valid_ratio=0.2, test_ratio=0.1)

Now, we are ready to annotate the all the images to create training, validation and testing data we will use for training our YOLO model. To do so we will be using free online annotation tool **LabelImg**. It is super easy to use, here is a quick tutorial [here](https://www.youtube.com/watch?v=VsZvT69Ssbs). Make sure to switch YOLO format at the beginning. Annotate each images in train,valid and test images in *root_dir/X/images* folders and save them to respective *root_dir/X/labels* folder. Simply use "droplet" as a default label name in the app.


Just run the following cell to access its user interface. In the app, zoom-in to droplets to create high quality bounding boxes. We have already done this before. Inspect the folders inside each directory. Notice that each text file in *"labels"* folder has exactly the same name with its corresponding image file in *"images"* folder.


If you have multiple experiments to carry out, check out Sec-6 in this notebook to get an idea about approximately how many training images you should use. You can save up quite a bit annotation time with that approach. 

In [ ]:
%run -i labelImg/labelImg.py

## Traning YOLO

- If we are done with annotation, we are ready to train and test our YOLOv8 model. We already annotated "three droplet" experiment, we will use the very same files here. If you are here just to see how this notebook works, copy train-valid-test folders within *datasets/three_droplet* folder in the main page and paste them into "tutorial_data/datasets". 

- The following option for model training is self-explanatory. We will save all the YOLO tranining results into *"project/name"* folder. In the same folder, you will find tons of useful information. The ones we will definetely use is /weights/best.pt' which is the best model of our tranining. We will load and use the best model in the rest of the notebook. 


- You can experiment with different pretrained models; yolov8n, yolov8s, yolov8m, yolov8l, yolov8x(increasing in size )

In [ ]:
#main yaml directory
data = 'datasets/sample.yaml'

#save all yolo results here
yolo_results = 'yolo_results'

#save tranining results to yolo_results/experiment_name
experiment_name = "sample_project"

#overwrite the traning results for different trials
exist_ok = True

#number of epochs
epochs = 1

#reproducibility
seed = 0

In [ ]:
#load and train the model
model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt') 

model.train(data=data, epochs=epochs,project=yolo_results,name=experiment_name,exist_ok=exist_ok)

In [ ]:
#test the model on testing images
data_test = 'datasets/dummy_test.yaml'
model.val(data = data_test)

- **If *mAP50* value on testing data is way below 0.90, that means the model did not learn enough thus will likely to fail in real-time tracker. Based on our experience, first start by increasing the the number of epochs to a high number say 400. If the behaviour is the same, increasing the number of training images may help. Add 20-30 images/labels to the training data. You can also increase the batch_size**. 

## Droplet/Intruder Tracking

- Once the model is trained, you will find the best model at  f"{project}/{name}/weights/best.pt" as noted above. To visualize and save the tracking results, simply modify the following cell. **track_droplet** displays the tracking in real-time and saves the trajectories to *save_dir/save_name.csv* which we will discuss in a bit. Ignore if you get "QObject::moveToThread" error. The same function also saves the tracking video in the same directory.


- Make sure to properly enter all the arguments. If you spot any false positives, try increasing the threshold slightly. You can always interrupt the simulation by pressing "q" on your keyboard. All information is saved in real-time.

In [ ]:
model_path = f"{yolo_results}/{experiment_name}/weights/best.pt"
model = YOLO(model_path) 

In [ ]:
#number of particles in the experiment
num_particle = 3

#experiment video to be tracked, defined at the top
video_path = video_path

#accept detections only above this confidance
conf_thresold = 0.45

#save tracks and tracking video here
save_dir = 'tracking_results'

#name your experiment
save_name = experiment_name

#show trajectory or just bounding box with IDs
show_trace = True

# #run the tracker
track_droplet(model=model, num_particle=num_particle, video_path=video_path, conf_thresold=conf_thresold,
                                        save_dir=save_dir, save_name=save_name, show_trace=show_trace)

                    experiment: sample_project detection_rate: 8909/9033 = 98.627%
                    trajectories saved to tracking_results/sample_project.csv

                    (8909, 9033, 185.93071365356445)

## Inspecting Results and Some Post-Processing

- Let's inspect the results regarding our original experiment.We will start by loading the dataframe we saved. You can analyze this data in a way you wish. 


- **frame_id, time, x, y, c** columns refers to the frame number, time stamp(sec), x-position,y-position of each individual droplet/intruder tracked. detected=1 means we detected precisely 3 particles(which is different in other experiments) in that frame with confidance score 0.45 we set above. This ensures we dont get false positives. detected=0 rows has only frame_id and time properties. It can be useful for diagnosis purposes. 


- For example, using the plot function below, we can overlay the position and the flow of the object.

In [ ]:
data = pd.read_csv(f"{save_dir}/{save_name}.csv")
data.head()

In [ ]:
df_with_speed = add_speed(data=data,num_particle=3)
df_with_speed.head()

In [ ]:
#square: inital point; circle:terminal point, you can enter num_particle=3 to see all speed maps
plot_speed(data=data,num_particle=1)

## Use Different Trackers with YOLOv8

- Any tracking-by-detection method can operate on YOLOv8 detection. [yolov8_tracker](https://github.com/mikel-brostrom/yolov8_tracking) by mikel-brostrom is an amazing tool to directly obtain the tracks from SOTA trackers on top of YOLOv8. As fas as I can see, 'strongsort', 'deepocsort', 'ocsort', 'bytetrack' and 'botsort' are supported there. Following is a simple implemantation. You can experiment with different tracker. 


- As we discussed in our paper, **these models suffer from multiple ID switches in all multiple droplet experiments. Thus, their results cannot be used by any means for multiple walking droplet experiments**. However, they can be useful for single particle tracking or some other experiments. It is not a good practice to run the command line arguments from Jupyter but this is just a demo. 

- Restart the notebook if you encounter any error in the following cell.

In [ ]:
# List of tracking methods ['strongsort', 'deepocsort', 'ocsort', 'bytetrack', 'botsort']
tracking_method = 'strongsort'

# Root directory for all tutorial data
track_dir = 'sota_tracker'

# Accept detections above this
conf_thresold = 0.45

#experiment name
exp_name = 'strongsort_track'

video_path = video_path

os.system(f"python ../yolov8_tracking/track.py --yolo-weights {model_path} --tracking-method {tracking_method}\
          --source {video_path} --conf-thres {conf_thresold} \
          --project {track_dir} --name {exp_name} \
          --show-vid --save-txt --save-vid \
          ")


                                               THANK YOU FOR CHECKING OUT!